# ArcGIS Online Organization Administration at Scale
## ESRI EdUC 2019
## Seth Peery, Sr. GIS Architect, Virginia Tech


In [1]:
# This simply sets up the environment for us to run the Jupyter Notebook as a presentation
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

{'height': 768, 'scroll': True, 'width': 1024}

This presentation is given as a jupyter notebook. Jupyter notebooks are a web-based interactive Python development environment.
![Architecture](jupyterarch.png)

# Rationale for org management
* Software as a Service products must be managed with the same attention we give on-premises systems
* Management objective is to lower impediments to use of AGO
* AGOL depends on finite shared resources; org administration is the stewardship of these resources
 * named users
 * service credits
 * Pro and other licenses
 * usability and organization of the site
* AGOL orgs can become unwieldy at scale
* User lifecycle stages require different management practices over time
* Processes, training and automation become increasingly important for large orgs


# Org administration tasks
* Enterprise Logins
* Auto provisioning
* Initial default privileges and entitements
* Credit stewardship
* Pro Licenses
* Ad hoc requests
* Content migration
* Deprovisioning

![events](adminflow.png)

## The Python API for organization administration
The python API provides a set of objects for administering your Web GIS.
See 
![GIS Module](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_gis_module_01.png)

## Connecting to your Web GIS (ArcGIS Online / ArcGIS Enterprise)
To connect to your organization, we import the requisite libraries and then create an object of type "GIS":

In [2]:
from arcgis import *
import requests
import time
import csv
import json
import pandas
from time import strftime

### Syntax to connect to ArcGIS Online
` gis = GIS("https://myorganization.maps.arcgis.com",username="An_admin_user",password="Please_do_not_put_this_in_clear_text")   ` 

In my case I put org-specific stuff into a config file so this notebook can be more easily shared with others.
The file looks like this:

`{
    "agolOrg":{
            "url":"https://yourOrgShortName.maps.arcgis.com",
            "username":"*******,
            "password":"*******",
            "shortName":"yourOrgShortName"
    },
    "authService":{
            "url":"https://some_url_that_checks_usernames",
            "username":"*****",
            "password":"*****",
            "valueIfTrue":"true"
    }
}`

### Making the Connection to your Org

In [3]:

# Then load it up like this
with open('vtActiveConfig.json') as configFile:
    myConfig = json.load(configFile)


# now connect
gis = GIS(myConfig['agolOrg']['url'],username=myConfig['agolOrg']['username'],password=myConfig['agolOrg']['password'])    

# verify that it works
try:
    org = gis.properties.name
    print ("Connected to " + org)
except exception as ex:
        print ("Error retrieving AGOL org properties.")

Connected to Virginia Tech


Now let's get some info about our users.

In [15]:
userList = []
users = gis.users.search(max_users=2000)

for user in users:
    #These things come straight from the user dict
    d_esriUsername = user.username
    d_fullName = user.fullName
    d_email = user.email
    d_role = user.role
    d_storage = (user.storageUsage / 1024)
    
    #number of content items <=100 is returned by length of items arr
    d_items = len(user.items())
    #print(d_items)
    
    #VT PID is returned by stripping off the _virginiatech
    d_pid = user.username.rsplit("_"+myConfig['agolOrg']['shortName'])[0]
    
    #last access comes from https://developers.arcgis.com/python/guide/accessing-and-managing-users/
    t_last_accessed = time.localtime(user.lastLogin/1000)
    d_lastAccess = "{}/{}/{}".format(t_last_accessed[0], t_last_accessed[1], t_last_accessed[2])
    
    #count of groups this user is a member of
    d_groupCount = len(user.groups)
    
    #Now build a data structure    
    currentUserInfo = {"pid":d_pid,
                        "esriUsername":d_esriUsername,
                        "fullName":d_fullName,
                        "email":d_email,
                        "storage":d_storage,
                        "role":d_role,
                        "lastAccess":d_lastAccess,
                        "groups":d_groupCount,
                        "items":d_items}
    userList.append(currentUserInfo)
    
# iteration done.
# now let's make a dataframe.  We'll use this later.
df = pandas.DataFrame(userList)
df




,email,esriUsername,fullName,groups,items,lastAccess,pid,role,storage
0,,aabhi_virginiatech,Abhishek Abhyankar,1,14,2016/5/4,aabhi,org_publisher,1399.333008
1,,abby15_virginiatech,Abby Qualliotine,1,2,2016/4/5,abby15,wEn8NO3VNLcycmpV,2006.706055
2,abbyb99@vt.edu,abbyb99_virginiatech,Abigail Bryerton,1,0,2018/3/11,abbyb99,pH1lPndPVtVbrE6l,0.000000
3,abbye97@vt.edu,abbye97_virginiatech,Abigail England,1,2,2017/4/18,abbye97,org_user,592.114258
4,abbyp98@vt.edu,abbyp98_virginiatech,Abigail Potter,1,15,2018/5/1,abbyp98,org_publisher,42183.235352
5,abdulm6@vt.edu,abdulm6_virginiatech,Abdulmueen Bogis,0,0,2018/4/22,abdulm6,pH1lPndPVtVbrE6l,0.000000
6,abelac@vt.edu,abelac_virginiatech,Ana Abel,1,7,2018/6/19,abelac,pH1lPndPVtVbrE6l,30911.347656
7,,abigailt_virginiatech,Abigail Thompson,1,4,2016/4/7,abigailt,wEn8NO3VNLcycmpV,2739.997070
8,ablood@vt.edu,ablood_virginiatech,Amy Blood,1,0,2018/6/13,ablood,org_user,0.000000
9,abreed2@vt.edu,abreed2_virginiatech,Alyson Breeding,0,0,2016/9/8,abreed2,org_user,0.000000


Now that we have a data structure of user information in memory, we can make administrative decisions based on it.

* Search for a role and grant its members ArcGIS Pro licenses
* Delete "drive by" users
* Identify users not affiliated with the institution using an out-of-band lookup service

In [9]:
# What are these weird custom roles?   Let's find out
roles = gis.users.roles.all()
for role in roles:
    print(role)

<Role name: Viewer, description: Viewer>
<Role name: WebMapping2016, description: Web Mapping Class, Spring 2016 led by Peter Sforza>
<Role name: VT-Dymond-Stormwater, description: Randy Dymond Stormwater = Publisher_Faculty + Tiles>
<Role name: Default, description: Grant this role to new AGOL users, escalating privileges only if necessary.>
<Role name: Analyst, description: Role for users that need to consume ESRI premium content, but not create objects.>
<Role name: Publisher-Faculty, description: Custom publisher role for VT Faculty>
<Role name: UAP5114, description: Custom role for Steve Chozick UAP 5114 class>
<Role name: EntomologyLab, description: Role for Entomology Lab students>
<Role name: Summer_Bridge, description: Summer Bridge Students>
<Role name: PesticideUsage, description: Mike Weavers Pesticide Usage Class>
<Role name: New_User, description: Default role to assign to newly auto-provisioned users; flags for processing>
<Role name: DigitalPlanet, description: Members 

In [6]:
gis.users.roles.get_role('pH1lPndPVtVbrE6l')

<Role name: New_User, description: Default role to assign to newly auto-provisioned users; flags for processing>

In [7]:
# Let's get all the users whose role is "New User".
# We're using the pandas query syntax here
df.loc[(df['role'] == 'pH1lPndPVtVbrE6l')]

,email,esriUsername,fullName,groups,items,lastAccess,pid,role,storage


In [7]:
# Let's give our new users an ArcGIS Pro license
# For now, we'll use the licenses and entitlements we expect
entitlements = {'ArcGIS Pro': ['geostatAnalystN', 'spatialAnalystN', 'networkAnalystN', 'dataReviewerN',
                               'dataInteropN', 'workflowMgrN', '3DAnalystN', 'desktopAdvN']}
licenses = {lic: gis.admin.license.get(lic) for lic in entitlements}
new_users = gis.users.search("pH1lPndPVtVbrE6l")            
for u in new_users:
    for license_type in entitlements:
        lic = licenses[license_type]
        # THIS IS A DEMO SO WE DON'T ACTUALLY PULL THE TRIGGER
        #lic.assign(username=u.username, entitlements=entitlements[license_type])
        print('{0} entitlements granted to user {1.username}'.format(license_type, u))

ArcGIS Pro entitlements granted to user abbyb99_virginiatech
ArcGIS Pro entitlements granted to user abdulm6_virginiatech
ArcGIS Pro entitlements granted to user abelac_virginiatech
ArcGIS Pro entitlements granted to user ack98_virginiatech
ArcGIS Pro entitlements granted to user adamwise_virginiatech
ArcGIS Pro entitlements granted to user aives95_virginiatech
ArcGIS Pro entitlements granted to user ajake5_virginiatech
ArcGIS Pro entitlements granted to user ajm13_virginiatech
ArcGIS Pro entitlements granted to user alerman5_virginiatech
ArcGIS Pro entitlements granted to user alexbb98_virginiatech
ArcGIS Pro entitlements granted to user alexn9_virginiatech
ArcGIS Pro entitlements granted to user ali17_virginiatech
ArcGIS Pro entitlements granted to user allym7_virginiatech
ArcGIS Pro entitlements granted to user alop_virginiatech
ArcGIS Pro entitlements granted to user amandar4_virginiatech
ArcGIS Pro entitlements granted to user amegna99_virginiatech
ArcGIS Pro entitlements granted 

## Example 2: Drive by users

In [5]:
#Let's just look for the users who have no content items or storage, nor are they in any groups.
driveBy = df.loc[(df['storage'] == 0) & (df['items'] == 0) &(df['groups'] ==0)]
driveBy

,email,esriUsername,fullName,groups,items,lastAccess,pid,role,storage
5,abdulm6@vt.edu,abdulm6_virginiatech,Abdulmueen Bogis,0,0,2018/4/22,abdulm6,pH1lPndPVtVbrE6l,0.0
9,abreed2@vt.edu,abreed2_virginiatech,Alyson Breeding,0,0,2016/9/8,abreed2,org_user,0.0
10,ach2058@vt.edu,ach2058_virginiatech,Austin Hayes,0,0,2018/7/20,ach2058,pH1lPndPVtVbrE6l,0.0
13,acorrea@vt.edu,acorrea_virginiatech,Angela Correa-Becker,0,0,2017/9/25,acorrea,org_publisher,0.0
16,adamwise@vt.edu,adamwise_virginiatech,Adam Wise,0,0,2018/4/29,adamwise,pH1lPndPVtVbrE6l,0.0
19,afb143@vt.edu,afb143_virginiatech,Ansel Bateman,0,0,2018/2/16,afb143,org_admin,0.0
23,aguest@vt.edu,aguest_virginiatech,Alex Guest,0,0,2017/4/7,aguest,org_user,0.0
25,aives95@vt.edu,aives95_virginiatech,Alexandra Ives,0,0,2018/4/25,aives95,pH1lPndPVtVbrE6l,0.0
28,ajwells@vt.edu,ajwells_virginiatech,Alex Wells,0,0,2016/12/20,ajwells,org_user,0.0
29,,akarpa1_virginiatech,Andrew Karpa,0,0,2016/12/14,akarpa1,org_user,0.0


In [6]:
len(df)

100

In [15]:
# Sort by last accessed time.
driveBy.sort_values('lastAccess')

,email,esriUsername,fullName,groups,items,lastAccess,pid,role,storage
28,,akarpa1_virginiatech,Andrew Karpa,0,0,2016/12/14,akarpa1,org_user,0.0
27,ajwells@vt.edu,ajwells_virginiatech,Alex Wells,0,0,2016/12/20,ajwells,org_user,0.0
95,asp2340@vt.edu,asp2340_virginiatech,Andrew Pierce,0,0,2016/7/25,asp2340,KeUYufHBaBEQkoZq,0.0
77,apagan8@vt.edu,apagan8_virginiatech,Alexis Pagan,0,0,2016/7/26,apagan8,org_user,0.0
96,augusta@vt.edu,augusta_virginiatech,Abigail August,0,0,2016/9/6,augusta,org_user,0.0
38,alext10@vt.edu,alext10_virginiatech,Alex Tucker,0,0,2016/9/6,alext10,org_user,0.0
41,aliyahli@vt.edu,aliyahli_virginiatech,Aliyah Linatoc,0,0,2016/9/8,aliyahli,org_user,0.0
9,abreed2@vt.edu,abreed2_virginiatech,Alyson Breeding,0,0,2016/9/8,abreed2,org_user,0.0
97,aupdike@vt.edu,aupdike_virginiatech,Aaron Updike,0,0,2017/10/2,aupdike,org_publisher,0.0
74,anratliff@vt.edu,anratliff@vt.edu,Andy Ratliff,0,0,2017/12/6,anratliff@vt.edu,org_admin,0.0


####  I feel reasonably confident we can get rid of users who 
* own no content items
* are members of no groups
* use no storage
* have not logged in for a year

... since if that user logs back in, it will be like they never left.

In [7]:
# SO we nuke them from orbit
deleteList = df.loc[(df['storage'] == 0) & (df['items'] == 0) &(df['groups'] ==0) &(df['lastAccess'].str[:4] != '2018')]
deleteList

,email,esriUsername,fullName,groups,items,lastAccess,pid,role,storage
9,abreed2@vt.edu,abreed2_virginiatech,Alyson Breeding,0,0,2016/9/8,abreed2,org_user,0.0
13,acorrea@vt.edu,acorrea_virginiatech,Angela Correa-Becker,0,0,2017/9/25,acorrea,org_publisher,0.0
23,aguest@vt.edu,aguest_virginiatech,Alex Guest,0,0,2017/4/7,aguest,org_user,0.0
28,ajwells@vt.edu,ajwells_virginiatech,Alex Wells,0,0,2016/12/20,ajwells,org_user,0.0
29,,akarpa1_virginiatech,Andrew Karpa,0,0,2016/12/14,akarpa1,org_user,0.0
32,alemair@vt.edu,alemair_virginiatech,Alison Willebeek-Lemair,0,0,2017/4/18,alemair,org_user,0.0
33,alena5@vt.edu,alena5_virginiatech,Alena Deveau,0,0,2017/4/20,alena5,org_publisher,0.0
38,alexj94@vt.edu,alexj94_virginiatech,Alex Johnston,0,0,2017/6/12,alexj94,org_publisher,0.0
40,alext10@vt.edu,alext10_virginiatech,Alex Tucker,0,0,2016/9/6,alext10,org_user,0.0
43,aliyahli@vt.edu,aliyahli_virginiatech,Aliyah Linatoc,0,0,2016/9/8,aliyahli,org_user,0.0


In [8]:
len(deleteList)

13

In [ ]:
for index, row in deleteList.iterrows():
    sUserToDelete = df['esriUsername'].values[index]
    print ("Deleting " + sUserToDelete +"...")
    #Here we would simply call
    #userToDelete = gis.users.search(sUserToDelete, max_users=1)
    #userToDelete.delete()

## Out of band affiliation check
Since ArcGIS Online cannot pull extended attributes from a SAML identity provider beyond username and email, we developed a web service to perform a check for "Active" status for any username we provide it.  

NOTE that this does not translate outside of the Virginia Tech context; we developed a custom solution for this.  
Your custom web services can be integrated... in orgConfig.json, provide values for the "authService" group:
* URL
* authKey or password to access the service
* value to be returned if the user is a valid member

In [13]:
userList = []
users = gis.users.search(max_users=2000)

# Wrap the call to the REST service in a function
def isActiveVT(pid):
    result = False
    url = myConfig['authService']['url']
    params = {'authkey':myConfig['authService']['password'],'pid':pid}
    r = requests.post(url,data=params)
    if(r.text==myConfig['authService']['valueIfTrue']):
        result=True
    return result

for user in users:
    d_esriUsername = user.username
    d_fullName = user.fullName
    d_email = user.email
    d_pid = user.username.rsplit("_"+myConfig['agolOrg']['shortName'])[0]

    # Call the custom REST service 
    d_active = "false"
    if(isActiveVT(d_pid)):
        d_active = myConfig['authService']['valueIfTrue'] 
    
    currentUserInfo = {"pid":d_pid,
                        "esriUsername":d_esriUsername,
                        "fullName":d_fullName,
                        "email":d_email,
                        "active":d_active}
    userList.append(currentUserInfo)
print (str(len(userList)) +" users are no longer VT affiliates.")
    
# iteration done.
# now let's make a dataframe.  We'll use this later.
df = pandas.DataFrame(userList)
df

1426 users are no longer VT affiliates.


,active,email,esriUsername,fullName,pid
0,false,,aabhi_virginiatech,Abhishek Abhyankar,aabhi
1,true,,abby15_virginiatech,Abby Qualliotine,abby15
2,true,abbyb99@vt.edu,abbyb99_virginiatech,Abigail Bryerton,abbyb99
3,true,abbye97@vt.edu,abbye97_virginiatech,Abigail England,abbye97
4,true,abbyp98@vt.edu,abbyp98_virginiatech,Abigail Potter,abbyp98
5,true,abdulm6@vt.edu,abdulm6_virginiatech,Abdulmueen Bogis,abdulm6
6,true,abelac@vt.edu,abelac_virginiatech,Ana Abel,abelac
7,false,,abigailt_virginiatech,Abigail Thompson,abigailt
8,true,ablood@vt.edu,ablood_virginiatech,Amy Blood,ablood
9,true,abreed2@vt.edu,abreed2_virginiatech,Alyson Breeding,abreed2


In [14]:
nonAffiliates = df.loc[(df['active'] == 'false') ]
print(str(len(nonAffiliates)) + " users are no longer VT Affiliates.")
nonAffiliates

240 users are no longer VT Affiliates.


,active,email,esriUsername,fullName,pid
0,false,,aabhi_virginiatech,Abhishek Abhyankar,aabhi
7,false,,abigailt_virginiatech,Abigail Thompson,abigailt
12,false,acline99@vt.edu,acline99_virginiatech,Adam Cline,acline99
22,false,jfmunsel@vt.edu,agroforestry,John Munsell,agroforestry
30,false,,alankt_virginiatech,Alan Thibault,alankt
33,false,alena5@vt.edu,alena5_virginiatech,Alena Deveau,alena5
35,false,alexabracale@vt.edu,alexabracale_virginiatech,Alexa Bracale,alexabracale
43,false,aliyahli@vt.edu,aliyahli_virginiatech,Aliyah Linatoc,aliyahli
44,false,allis95@vt.edu,allis95_virginiatech,Allison Guzman,allis95
48,false,alyssac@vt.edu,alyssac_virginiatech,Alyssa Collins,alyssac


# Automation needs to integrate with your business processes 
* Once we generate a list of users who need to be deprovisioned, then what?
 * We could send them an automatic e-mail notification
 * Provide references for self-deprovisioning (AGO-Assistant)
 * Set a time limit for automated content migration
   * to another user
   * to an orphaned content account
* Note that you can't delete users with remaining content.
* Content management is a whole separate presentation



![ago-assist](agoassist.png)

# Options for running Python API code in production
* Interactively 
 * Jupyter noteboook
 * Your preferred Python IDE
 * Command line
* Event driven
 * AWS Lambda
* Recurring
 * AWS Lambda
 * cron job/ scheduled task

![lambda](lambda.png)

# Link to this Presentation and Code
![qr](qr.png)
https://github.com/sspeery/esri-seuc-2018 



# Presenter Contact Information
>*Seth Peery*  
>    Senior GIS Architect  
>    Enterprise GIS (0214)  
>        1700 Pratt Drive  
>        Blacksburg, VA 24061  
>    (540) 231-2178  
>    sspeery@vt.edu   
>    http://gis.vt.edu